# Coursera Capstone Project

## The Battle of Neighborhoods - Week 1

#### *RIZA YILDIRIM*

<br>

## INTRODUCTION

#### CASE:

#### I am living in Istanbul / Turkey. My house is within walking distance to metro and bus stations and I can find many different things in the area, such as various cousine restaurants, cafes, food shops and entertainment. I have been offered a great opportunity to work for a leader firm in Brooklyn, NY. I am very excited and I want to use this opportunity. The key question is : How can I find a convenient and enjoyable place similar to mine now in Istanbul? In order to make a comparison and evaluation of the rental options in Brooklyn NY, I must set some basis, therefore the apartment in Manhattan must meet the following demands:

- apartment must be 2 or 3 rooms
- desired location is near a metro station in the Brooklyn area and within 1.5 km (1 mile) radius
- price of rent not exceed $5,000 per month
- top places in the selected neighborhood shall be similar to current residence
- desirable to have venues such as coffee shops, Turkish restaurants, wine stores, gym and food shops

## DATA SECTION

### Description: 
The following data is required to answer the issues of the problem:

- List of Boroughs and neighborhoods of Brooklyn with their geodata
- List of Subway metro stations in Brooklyn with their address location
- List of apartments for rent in Brooklyn area with their addresses and price
- Preferably, a list of apartment for rent with additional information, such as price, address, area, # of beds, etc
- Venues for each Brooklyn neighborhood (than can be clustered)
- Venues for subway metro stations, as needed

### How the data will be used to solve the problem
The data will be used as follows:

- Use Foursquare and geopy data to map top 10 venues for all Manhattan neighborhoods and clustered in groups ( as per Course LAB)
- Use foursquare and geopy data to map the location of subway metro stations , separately and on top of the above clustered map in order to be able to identify the venues and ammenities near each metro station, or explore each subway location separately
- Use Foursquare and geopy data to map the location of rental places, in some form, linked to the subway locations.
- Create a map that depicts, for instance, the average rental price per square ft, around a radious of 1.0 mile (1.6 km) around each subway station - or a similar metrics. I will be able to quickly point to the popups to know the relative price per subway area.
- Addresses from rental locations will be converted to geodata( lat, long) using Geopy-distance and Nominatim.
- Data will be searched in open data sources if available, from real estate sites if open to reading, libraries or other government agencies such as Metro New York MTA, etc.

The procesing of these DATA will allow to answer the key questions to make a decision:

- what is the cost of rent (per square ft) around a mile radius from each subway metro station?
- what is the area of Manhattan with best rental pricing that meets criteria established?
- What is the distance from work place ( Park Ave and 53 rd St) and the tentative future home?
- What are the venues of the two best places to live? How the prices compare?
- How venues distribute among Manhattan neighborhoods and around metro stations?
- Are there tradeoffs between size and price and location?
- Any other interesting statistical data findings of the real estate and overall data.

In [1]:
!conda install -c conda-forge geopy --y
import numpy as np # library to handle data in a vectorized manner
import time
import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
import random # library for random number generation
import json # library to handle JSON files
import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe
# libraries for displaying images
from IPython.display import Image 
from IPython.core.display import HTML 
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values
!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
!conda install -c anaconda beautifulsoup4 --y
!conda install -c anaconda lxml --y


import folium # map rendering library

print('Libraries imported.')
print('Folium installed')


Solving environment: done

## Package Plan ##

  environment location: /opt/conda/envs/Python36

  added / updated specs: 
    - geopy


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    geopy-1.20.0               |             py_0          57 KB  conda-forge
    ca-certificates-2019.11.28 |       hecc5488_0         145 KB  conda-forge
    openssl-1.1.1d             |       h516909a_0         2.1 MB  conda-forge
    geographiclib-1.50         |             py_0          34 KB  conda-forge
    certifi-2019.11.28         |           py36_0         149 KB  conda-forge
    ------------------------------------------------------------
                                           Total:         2.5 MB

The following NEW packages will be INSTALLED:

    geographiclib:   1.50-py_0         conda-forge
    geopy:           1.20.0-py_0       conda-forge

The following packages will be UPDATED:

    ca-

### Reference of venues around current residence in Istanbul for comparison to Brooklyn

In [ ]:
# Istanbul adress
address = 'gumuscu sk, bostanci, 34744'
geolocator = Nominatim(user_agent="foursquare_agent")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Istanbul home are {}, {}.'.format(latitude, longitude))


In [ ]:
LIMIT = 100 # limit of number of venues returned by Foursquare API
radius = 500 # define radius
CLIENT_ID = 'XM5S5YSHAFKHXPWUZEVU5UFYYM2UVPYEOXBOZ1HVLDVUSO1D' # my Foursquare ID
CLIENT_SECRET = 'BM3DYWQNSIPRKXNJGKPPZK2WSW2BTAFGSJSPSIL4J2XQ4HLZ' # my Foursquare Secret
VERSION = '20190604'

# create URL
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    latitude, 
    longitude, 
    radius, 
    LIMIT)
url # display URL

In [ ]:
results = requests.get(url).json()
#results

In [ ]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [ ]:
venues = results['response']['groups'][0]['items']
    
SGnearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
SGnearby_venues =SGnearby_venues.loc[:, filtered_columns]

# filter the category for each row
SGnearby_venues['venue.categories'] = SGnearby_venues.apply(get_category_type, axis=1)

# clean columns
SGnearby_venues.columns = [col.split(".")[-1] for col in SGnearby_venues.columns]

SGnearby_venues.head(10)

In [ ]:
# create map of my house in Istanbul using latitude and longitude values
map_sg = folium.Map(location=[latitude, longitude], zoom_start=20)

# add markers to map
for lat, lng, label in zip(SGnearby_venues['lat'], SGnearby_venues['lng'], SGnearby_venues['name']):
    label = folium.Popup(label, parse_html=True)
    folium.RegularPolygonMarker(
        [lat, lng],
        number_of_sides=4,
        radius=10,
        popup=label,
        color='blue',
        fill_color='#0f0f0f',
        fill_opacity=0.7,
    ).add_to(map_sg)  
    
map_sg